In [ ]:
!pip install pytorch_lightning

import pandas as pd
from transformers import AutoModel, AutoTokenizer
from google.colab import drive
drive.mount('/content/drive')
import numpy as np

%pip install camel-tools

from pathlib import Path
S31_DB_PATH = Path('../data/disambig_db/calima-msa-s31.db')
S31_DB = MorphologyDB(S31_DB_PATH, 'a')
S31_AN = Analyzer(S31_DB, 'NOAN_ALL', cache_size=100000)
bert_disambig = BERTUnfactoredDisambiguator.pretrained('msa', top=1000, pretrained_cache = False)
bert_disambig._analyzer = S31_AN


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for camel-kenlm: filename=camel_kenlm-2023.3.17.2-cp310-cp310-linux_x86_64.whl size=3453124 sha256=df9daf9bc6a95e57542e04f4892d2bdd25fcdc3b51f38d8feef9c6b47c60aa3f
  Stored in directory: /root/.cache/pip/wheels/29/c5/32/09633c3b70fdfc470b2fb912bd9e90d8d6814df68c794dcaa6
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=e3ca6c1d6d0d

Some weights of the model checkpoint at /content/drive/MyDrive/camel_tools/data/disambig_bert_unfactored/msa were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Objective: on the ALIGNED-tagged corpus, get counts of all POS x Level


In [6]:
# aligned fragments data import

frag_train = pd.read_csv('../data/all_train_aligned.csv')
frag_dev = pd.read_csv('../data/all_dev_aligned.csv')
frag_test = pd.read_csv('../data/all_test_aligned.csv')


frag_train = frag_train[frag_train.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_dev = frag_dev[frag_dev.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_test = frag_test[frag_test.apply(lambda x: type(x['0']) == str, axis = 1)]

def sort_score(list_of_analyses):
  list_of_analyses.sort(key = lambda x: x.score, reverse = True)
  highest_score = list_of_analyses[0].score
  analyses_with_equal_score = [x for x in list_of_analyses
                                if x.score == highest_score]
  return analyses_with_equal_score

def score_select(list_of_analyses):
  list_of_analyses = sort_score(list_of_analyses)
  return list_of_analyses[0].analysis

def counts_pipeline(fragment):
  tokens = [t.split('#')[0] for t in fragment.split(' ')]
  gt_levels = [t.split('#')[1] for t in fragment.split(' ')]
  analyses = [token.analyses for token in bert_disambig.disambiguate(tokens)]
  picked_analyses = [score_select(analysis) for analysis in analyses]

  return [[t, g, a['pos']] for t, a, g in zip(tokens, picked_analyses, gt_levels)]


In [9]:
from tqdm import tqdm
all_counts = []
for x in tqdm(frag_train['0']):
  all_counts.extend(counts_pipeline(x))
for x in tqdm(frag_dev['0']):
  all_counts.extend(counts_pipeline(x))
for x in tqdm(frag_test['0']):
  all_counts.extend(counts_pipeline(x))

100%|██████████| 3274/3274 [04:55<00:00, 11.09it/s]


#### Pivot on pos x level.

In [10]:
all_counts_df = pd.DataFrame(all_counts)
summary = all_counts_df.groupby([1, 2]).size().reset_index(name='count')
pivot_table = summary.pivot(index=2, columns=1, values='count').fillna(0)

In [18]:
pivot_table.to_csv('../data/analysis/all_pos_aligned.csv')

### Calculate on Train only

In [35]:
all_counts_train = []
for x in tqdm(frag_train['0']):
  all_counts_train.extend(counts_pipeline(x))

100%|██████████| 14332/14332 [21:51<00:00, 10.93it/s]


In [36]:
all_counts_train_df = pd.DataFrame(all_counts_train)
summary_train = all_counts_train_df.groupby([1, 2]).size().reset_index(name='count')
pivot_table_train = summary_train.pivot(index=2, columns=1, values='count').fillna(0)

In [37]:
pivot_table_train